# Table 3. Descriptive Outcomes
On-scene survival (Yes=1, No=0)
ICU yes (Length >0)
If ICU yes, Length of ICU stay
Length of hospital stay
Survival of hospital stay
1 y survival


In [ ]:
import pandas as pd
from pediatric_trauma.utils import preprocess_diagnoses

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/data/Data_PedRegaTrauma_coded_for_analysis_250417.xlsx'

In [ ]:
df = pd.read_excel(data_path, sheet_name='All centres cleaned')

In [ ]:
df.head()

In [ ]:
df = preprocess_diagnoses(df)

In [ ]:
def create_table3(df):    
    assert df["Pat ID"].nunique() == len(df), "There are duplicate patient IDs in the dataset." 

    pop_df = pd.DataFrame()
    str_df = pd.DataFrame()

    # get the number of patients
    n_patients = len(df)
    pop_df['n_patients'] = [n_patients]
    str_df['n_patients'] = [n_patients]

    # On-scene survival (Yes=1, No=0)
    pop_df['n_on_scene_survival'] = [len(df[df['on-sceen survival (y/n)'] == 1])]
    pop_df['prop_on_scene_survival'] = [pop_df['n_on_scene_survival'].values[0] / n_patients]
    str_df['on_scene_survival'] = f'{pop_df["n_on_scene_survival"].values[0]} ({pop_df["prop_on_scene_survival"].values[0]:.1%})'

    # ICU yes (Length >0)
    df['length of ICU stay'] = df['length of ICU stay'].replace(999, pd.NA)
    pop_df['n_icu_yes'] = [len(df[df['length of ICU stay'] > 0])]
    pop_df['prop_icu_yes'] = [pop_df['n_icu_yes'].values[0] / n_patients]
    str_df['icu_admission'] = f'{pop_df["n_icu_yes"].values[0]} ({pop_df["prop_icu_yes"].values[0]:.1%})'

    # If ICU yes, Length of ICU stay
    with_icu_df = df[df['length of ICU stay'] > 0]
    # replace 999 with nan
    with_icu_df['length of ICU stay'] = with_icu_df['length of ICU stay'].replace(999, pd.NA)
    pop_df['icu_los_median'] = [with_icu_df['length of ICU stay'].median()]
    pop_df['icu_los_q1'] = [with_icu_df['length of ICU stay'].quantile(0.25)]
    pop_df['icu_los_q3'] = [with_icu_df['length of ICU stay'].quantile(0.75)]
    str_df['icu_los'] = f'{pop_df["icu_los_median"].values[0]} [{pop_df["icu_los_q1"].values[0]}-{pop_df["icu_los_q3"].values[0]}]'

    # Length of hospital stay
    # replace 999 with nan
    df['Length of hospital stay'] = df['Length of hospital stay'].replace(999, pd.NA)
    pop_df['hospital_los_median'] = [df['Length of hospital stay'].median()]
    pop_df['hospital_los_q1'] = [df['Length of hospital stay'].quantile(0.25)]
    pop_df['hospital_los_q3'] = [df['Length of hospital stay'].quantile(0.75)]
    str_df['hospital_los'] = f'{pop_df["hospital_los_median"].values[0]} [{pop_df["hospital_los_q1"].values[0]}-{pop_df["hospital_los_q3"].values[0]}]'
    
    # Survival of hospital stay
    df['Place discharge'] = df['Place discharge'].replace(999, pd.NA)
    pop_df['n_hospital_survival'] = [len(df[(df['Place discharge'] == 1) | (df['Place discharge'] == 2) | (df['Place discharge'] == 3) | (df['Place discharge'] == 4)])]
    pop_df['prop_hospital_survival'] = [pop_df['n_hospital_survival'].values[0] / n_patients]
    str_df['hospital_survival'] = f'{pop_df["n_hospital_survival"].values[0]} ({pop_df["prop_hospital_survival"].values[0]:.1%})'

    # 1 y survival
    df['1y survival'] = df['1y survival'].replace(999, pd.NA)
    pop_df['n_1y_survival'] = [len(df[df['1y survival'] == 1])]
    pop_df['prop_1y_survival'] = [pop_df['n_1y_survival'].values[0] / n_patients]
    str_df['1y_survival'] = f'{pop_df["n_1y_survival"].values[0]} ({pop_df["prop_1y_survival"].values[0]:.1%})'

    return pop_df.T, str_df.T

In [ ]:
pop_df, str_df = create_table3(df)
str_df

In [ ]:
dx_correct_pop_df, dx_correct_str_df  = create_table3(df[df['prehospital_diagnosis_correct'] == True])
dx_incorrect_pop_df, dx_incorrect_str_df = create_table3(df[df['prehospital_diagnosis_correct'] == False])

In [ ]:
joined_str_df = pd.concat(
    [str_df, 
    dx_correct_str_df.add_suffix(' (correct)'), 
    dx_incorrect_str_df.add_suffix(' (incorrect)')],
    axis=1
)

# column names: overall, correct pre-hospital diagnosis, incorrect pre-hospital diagnosis
joined_str_df.columns = ['overall', 'correct pre-hospital diagnosis', 'incorrect pre-hospital diagnosis']

joined_str_df

In [ ]:
# joined_str_df.to_csv('/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/data/table3.csv')